In [5]:
from datasets import load_dataset
from datasets import Split

In [8]:
import pandas as pd

In [2]:
train = pd.read_json('../data/poquad_train.json', lines=True).set_index('id')


In [3]:
import sentencepiece as spm

In [13]:
# Load allegro/plt5-large model from Hugging Face
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

tokenizer = AutoTokenizer.from_pretrained("allegro/plt5-large", use_fast=False)


model = AutoModelWithLMHead.from_pretrained("allegro/plt5-large")

/home/lazydart/miniconda3/envs/tensorflow/lib/python3.11/site-packages/transformers/models/auto/modeling_auto.py:1581: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [27]:
# Generate answers to questions from train["question"] based on train["context"]
def generate_answers(context, question):
    encoded_input = tokenizer(["Pytanie: " + question + " Kontekst: " + context + " </s>"],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
    
    output = model.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
    
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    # inputs = tokenizer(context, question, return_tensors="pt")
    # outputs = model(**inputs, decoder_input_ids=inputs["input_ids"])
    
    # logits = outputs.logits
    
    # all_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # answer_tokens = all_tokens[torch.argmax(logits[0]) : torch.argmax(logits[0]) + 1]
    # answer = tokenizer.decode(outputs)
    
    return output

# Generate answers for all questions in train
answers = train.iloc[:10].apply(lambda x: generate_answers(x["context"], x["question"]), axis=1)



In [34]:
i = 7
print(answers.iloc[i])

print(train.iloc[i].question)

print(train.iloc[i].context)

規. Model PD został szybko zastąpiony przez samoloty Pomilio PE.̋.誰
Jaki model zastąpił samoloty Pomilio PD?
Samoloty Pomilio PD weszły na wyposażenie jednostek bojowych i zarazem do walki od lipca 1917 roku. Pierwsze otrzymały je w tym miesiącu eskadry (Squadriglia) nr 131 i 132, gdzie zastąpiły Pomilio PC. Do działań wchodziły następnie kolejne nowo formowane eskadry Pomilio: 133 (we wrześniu), 134 (w październiku), 135 (w styczniu 1918 roku), 136 (w maju 1918 roku). Eskadra 139, sformowana z dwóch sekcji początkowo przydzielonych do lotnictwa Marynarki (od sierpnia 1917 roku), weszła do działań w styczniu 1918 roku, a w czerwcu odznaczyła się w bitwie nad Piawą, zrzucając 175 bomb. W bitwie tej brały udział też inne eskadry. Model PD został szybko zastąpiony w jednostkach przez PE i skierowany głównie do szkolenia.


In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Print the number of GPUs available and some details about them
    num_gpus = torch.cuda.device_count()
    print(f"CUDA is available. Number of GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. No GPU detected.")


CUDA is available. Number of GPUs: 1
GPU 0: NVIDIA GeForce RTX 3070 Laptop GPU


In [9]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

/bin/bash: /home/lazydart/miniconda3/envs/tensorflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [10]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric
from transformers import TrainerCallback

class PrintMemoryUsageCallback(TrainerCallback):
    """ Callback that prints memory allocation during training """
    def on_step_end(self, args, state, control, **kwargs):
        print(f"Step {state.global_step}: {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB allocated")

def preprocess_function(examples):
    inputs = ["pytanie: " + q + " kontekst: " + c for q, c in zip(examples['question'], examples['context'])]
    targets = [a['text'][0] if a['text'] else "nie wiadomo" for a in examples['answers']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")  # Dynamic padding

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")  # Dynamic padding

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

dataset = load_dataset('json', data_files={'train': '../data/poquad_train.json', 'validation': '../data/poquad_validation.json'})
model_name = "allegro/plt5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Reduced batch size
    per_device_eval_batch_size=1,  # Reduced batch size
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    gradient_accumulation_steps=4,  # Add gradient accumulation
    fp16=True  # Enable mixed precision
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    callbacks=[PrintMemoryUsageCallback()]  # Optional: To monitor memory usage
)

trainer.train()

model.save_pretrained('./finetuned_model')
tokenizer.save_pretrained('./finetuned_model')


tokenizer_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

...

  0%|          | 0/11546 [00:00<?, ?it/s]

Step 1: 1079.95 MB allocated
Step 2: 1079.95 MB allocated
Step 3: 1079.95 MB allocated
Step 4: 1079.95 MB allocated
Step 5: 1079.95 MB allocated
Step 6: 1079.95 MB allocated
Step 7: 1079.95 MB allocated
Step 8: 1079.95 MB allocated
Step 9: 1079.95 MB allocated
Step 10: 1079.95 MB allocated
Step 11: 1079.95 MB allocated
Step 12: 1079.95 MB allocated
Step 13: 1079.95 MB allocated
Step 14: 1079.95 MB allocated
Step 15: 1079.95 MB allocated
Step 16: 1079.95 MB allocated
Step 17: 1079.95 MB allocated
Step 18: 1079.95 MB allocated
Step 19: 1079.95 MB allocated
Step 20: 1079.95 MB allocated
Step 21: 1079.95 MB allocated
Step 22: 1079.95 MB allocated
Step 23: 1079.95 MB allocated
Step 24: 1079.95 MB allocated
Step 25: 1079.95 MB allocated
Step 26: 1079.95 MB allocated
Step 27: 1079.95 MB allocated
Step 28: 1079.95 MB allocated
Step 29: 1079.95 MB allocated
Step 30: 1079.95 MB allocated
Step 31: 1079.95 MB allocated
Step 32: 1079.95 MB allocated
Step 33: 1079.95 MB allocated
Step 34: 1079.95 MB

  0%|          | 0/5764 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_runtime': 160.0231, 'eval_samples_per_second': 36.02, 'eval_steps_per_second': 36.02, 'epoch': 1.0}
{'train_runtime': 4545.5633, 'train_samples_per_second': 10.161, 'train_steps_per_second': 2.54, 'train_loss': 3.145406322257886, 'epoch': 1.0}


('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/spiece.model',
 './finetuned_model/added_tokens.json')